In [1]:
#Import pandas, matplotlib.pyplot, and seaborn in the correct lines below
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
#from datetime import datetime

from library.sb_utils import save_file

In [2]:
# load the covid data for each county from https://www.kaggle.com/datasets/fireballbyedimyrnmom/us-counties-covid-19-dataset?resource=download
county_data = pd.read_csv('../data/us-counties.csv')

In [3]:
# Call info to get a summary of the data
county_data = county_data.fillna(0)
county_data['deaths'] = county_data['deaths'].astype(int)
county_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2502832 entries, 0 to 2502831
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   date    object 
 1   county  object 
 2   state   object 
 3   fips    float64
 4   cases   int64  
 5   deaths  int32  
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 105.0+ MB


`fips` refers to the fips county code. `cases` and `deaths` refers to the total number of cases / deaths total in each county (not for each day but a running total).

In [4]:
# Call head to get first few rows of data
county_data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


We can see that the our data is sorted by date order as Washington State had the first reported case of COVID-19 in the United States. We can also see that data begins recording for a specific county once they record their first COVID-19 case.

Looking at our other data source, we will not be using the fips county code. So we will drop the column.

In [5]:
#Drop the 'fips' column from county_data. Use inplace=True
county_data.drop(columns='fips', inplace=True)
county_data.head()

,date,county,state,cases,deaths
0,2020-01-21,Snohomish,Washington,1,0
1,2020-01-22,Snohomish,Washington,1,0
2,2020-01-23,Snohomish,Washington,1,0
3,2020-01-24,Cook,Illinois,1,0
4,2020-01-24,Snohomish,Washington,1,0


Count the number of missing values in each column and sort them.

In [6]:
#Count (using `.sum()`) the number of missing values (`.isnull()`) in each column of 
#ski_data as well as the percentages (using `.mean()` instead of `.sum()`).
#Order them (increasing or decreasing) using sort_values
#Call `pd.concat` to present these in a single table (DataFrame) with the helpful column names 'count' and '%'
missing = pd.concat([county_data.isnull().sum(), 100 * county_data.isnull().mean()], axis=1)
missing.columns=['Count', '%']
missing.sort_values(by='Count', ascending=False)

,Count,%
date,0,0.0
county,0,0.0
state,0,0.0
cases,0,0.0
deaths,0,0.0


This is good for our dataset as we see that we have no NaN values listed.

We now need to investigate counties with invalid names (Unkown).

In [7]:
#Remove rows with Unknown County Name
indexName = county_data[county_data['county'] == 'Unknown'].index
county_data.drop(indexName , inplace=True)
county_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2481431 entries, 0 to 2502831
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   date    object
 1   county  object
 2   state   object
 3   cases   int64 
 4   deaths  int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 104.1+ MB


We can see that our row total has dropped a significant amount since removing counties with an unknown name.

Do we want to keep county information from all states?

In [8]:
county_data['state'].value_counts()

Texas                       193498
Georgia                     124105
Virginia                    103089
Kentucky                     92530
Missouri                     89562
Kansas                       79048
Illinois                     78835
North Carolina               77808
Iowa                         76327
Tennessee                    74008
Indiana                      71881
Nebraska                     68736
Ohio                         68662
Minnesota                    67104
Michigan                     64424
Mississippi                  64079
Oklahoma                     59283
Arkansas                     58247
Puerto Rico                  57605
Wisconsin                    55858
Florida                      52590
Pennsylvania                 52439
Alabama                      52312
Louisiana                    50116
South Dakota                 49547
Colorado                     49463
New York                     45682
California                   45676
West Virginia       

We want to remove the states not officially apart of the United States except District of Columbia.

These states include Northern Mariana Islands, Virgin Islands, and Puerto Rico

In [9]:
#Remove rows with NMI, VI, and PR
indexState = county_data[(county_data['state'] == 'Northern Mariana Islands') | (county_data['state'] == 'Virgin Islands')
                         | (county_data['state'] == 'Puerto Rico')].index
county_data.drop(indexState , inplace=True)
county_data['state'].value_counts()

Texas                   193498
Georgia                 124105
Virginia                103089
Kentucky                 92530
Missouri                 89562
Kansas                   79048
Illinois                 78835
North Carolina           77808
Iowa                     76327
Tennessee                74008
Indiana                  71881
Nebraska                 68736
Ohio                     68662
Minnesota                67104
Michigan                 64424
Mississippi              64079
Oklahoma                 59283
Arkansas                 58247
Wisconsin                55858
Florida                  52590
Pennsylvania             52439
Alabama                  52312
Louisiana                50116
South Dakota             49547
Colorado                 49463
New York                 45682
California               45676
West Virginia            42234
Montana                  40654
North Dakota             39782
South Carolina           36088
Idaho                    33172
Washingt

In [10]:
#the number of unique values in each
county_data[['county', 'state']].nunique()

county    1850
state       51
dtype: int64

We now have 51 states/areas of the US 

We want to now add two columns to represent the change every day of covid cases as well as deaths

In [11]:
# add new column for change in cases and deaths per day
county_data['date'] = pd.to_datetime(county_data['date'], format="%Y-%m-%d")
groupdf = county_data.groupby('date')

def add(row):
    newdate = row['date'] - datetime.timedelta(days=1)
    try:
        datedf = groupdf.get_group(newdate)
        prev = datedf.loc[(datedf['county'] == row['county']) & (datedf['state'] == row['state'])]
        row['cases_change'] = row['cases'] - int(prev['cases'])
        row['deaths_change'] = row['deaths'] - int(prev['deaths'])
        return row
    except:
        row['cases_change'] = 0
        row['deaths_change'] = 0
        return row

county_data = county_data.apply(add, axis=1)
county_data.head()

,date,county,state,cases,deaths,cases_change,deaths_change
0,2020-01-21,Snohomish,Washington,1,0,0,0
1,2020-01-22,Snohomish,Washington,1,0,0,0
2,2020-01-23,Snohomish,Washington,1,0,0,0
3,2020-01-24,Cook,Illinois,1,0,0,0
4,2020-01-24,Snohomish,Washington,1,0,0,0


In [12]:
# Now lets work with our other data sources taken from https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data/
# the supplied CSV data file is the data directory
# demographics taken from https://data.census.gov/table?q=P6&t=Race+and+Ethnicity&g=010XX00US$0500000&y=2020&tid=DECENNIALDHC2020.P6
education = pd.read_csv('../data/education.csv')
population = pd.read_csv('../data/population_estimates.csv')
poverty = pd.read_csv('../data/poverty_estimates.csv')
demographics = pd.read_csv('../data/demographics.csv')

In [13]:
# Call info to get a summary of the data
education.info()
population.info()
poverty.info()
demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3285 entries, 0 to 3284
Data columns (total 55 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Federal Information Processing Standard (FIPS) Code                       3285 non-null   int64  
 1   State                                                                     3285 non-null   object 
 2   Area name                                                                 3285 non-null   object 
 3   2003 Rural-urban Continuum Code                                           3221 non-null   float64
 4   2003 Urban Influence Code                                                 3221 non-null   float64
 5   2013 Rural-urban Continuum Code                                           3221 non-null   float64
 6   2013 Urban Influence Code                                       

In [14]:
education.head()

,Federal Information Processing Standard (FIPS) Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Percent of adults completing some college or associate's degree, 2008-12","Percent of adults with a bachelor's degree or higher, 2008-12","Less than a high school diploma, 2017-21","High school diploma only, 2017-21","Some college or associate's degree, 2017-21","Bachelor's degree or higher, 2017-21","Percent of adults with less than a high school diploma, 2017-21","Percent of adults with a high school diploma only, 2017-21","Percent of adults completing some college or associate's degree, 2017-21","Percent of adults with a bachelor's degree or higher, 2017-21"
0,0,US,United States,NaN,NaN,NaN,NaN,"52,373,312","34,158,051","11,650,730",...,29.0,28.5,"25,050,356","59,636,386","64,656,741","75,808,834",11.1,26.5,28.7,33.7
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,"1,062,306","468,269","136,287",...,29.0,22.3,"430,047","1,041,209","1,032,122","910,425",12.6,30.5,30.2,26.7
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,"6,611","3,757",933,...,29.6,21.7,"4,126","12,977","11,367","11,144",10.4,32.8,28.7,28.1
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,"18,726","8,426","2,334",...,31.8,27.7,"14,555","44,342","50,518","52,562",9.0,27.4,31.2,32.5
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,"8,120","2,242",581,...,25.8,14.5,"4,378","6,600","5,010","2,007",24.3,36.7,27.8,11.2


In [15]:
population.head()

,FIPStxt,State,Area_Name,Rural_Urban_Continuum_Code_2003,Rural_Urban_Continuum_Code_2013,Urban_Influence_2003,Urban_Influence_2013,Economic_typology_2015,CENSUS_2020_POP,ESTIMATES_BASE_2020,...,R_DEATH_2021,R_DEATH_2022,R_NATURAL_CHG_2021,R_NATURAL_CHG_2022,R_INTERNATIONAL_MIG_2021,R_INTERNATIONAL_MIG_2022,R_DOMESTIC_MIG_2021,R_DOMESTIC_MIG_2022,R_NET_MIG_2021,R_NET_MIG_2022
0,0,US,United States,NaN,NaN,NaN,NaN,NaN,"331,449,281","331,449,520",...,10.4,10.4,0.4,0.7,1.1,3.0,0.0,0.0,1.1,3.0
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,NaN,"5,024,279","5,024,356",...,13.7,13.2,-2.4,-1.7,0.4,0.9,5.5,5.7,5.9,6.6
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,0.0,"58,805","58,802",...,11.5,11.3,0.2,0.6,0.3,0.5,4.1,8.5,4.3,9.0
3,1003,AL,Baldwin County,4.0,3.0,5.0,2.0,5.0,"231,767","231,761",...,12.9,12.0,-2.9,-2.1,0.4,1.3,29.5,28.7,29.9,30.0
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,3.0,"25,223","25,224",...,15.5,15.1,-4.3,-3.9,0.0,0.2,-12.4,9.9,-12.4,10.0


In [16]:
poverty.head()

,FIPS_Code,State,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_ 2013,POVALL_2021,CI90LBALL_2021,CI90UBALL_2021,...,CI90UB517P_2021,MEDHHINC_2021,CI90LBINC_2021,CI90UBINC_2021,POV04_2021,CI90LB04_2021,CI90UB04_2021,PCTPOV04_2021,CI90LB04P_2021,CI90UB04P_2021
0,0,US,United States,NaN,NaN,NaN,NaN,41393176.0,41149497.0,41636855.0,...,16.3,69717.0,69583.0,69851.0,3349149.0,3299669.0,3398629.0,18.3,18.0,18.6
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,800848.0,782169.0,819527.0,...,22.5,53990.0,53218.0,54762.0,71220.0,66888.0,75552.0,25.1,23.6,26.6
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,6296.0,4772.0,7820.0,...,20.4,66444.0,60061.0,72827.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,25526.0,21599.0,29453.0,...,18.5,65658.0,60723.0,70593.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,5089.0,3773.0,6405.0,...,44.6,38649.0,34308.0,42990.0,NaN,NaN,NaN,NaN,NaN,NaN


Lets first remove the unwanted rows (State Total Rows) as well as filter out the columns we want for education

In [17]:
education.dropna(subset = ['2003 Rural-urban Continuum Code'], inplace=True)
education = education[['State','Area name','Less than a high school diploma, 2017-21',
                       'High school diploma only, 2017-21', 'Some college or associate\'s degree, 2017-21',
                       'Bachelor\'s degree or higher, 2017-21',
                       'Percent of adults with less than a high school diploma, 2017-21',
                       'Percent of adults with a high school diploma only, 2017-21',
                       'Percent of adults completing some college or associate\'s degree, 2017-21',
                       'Percent of adults with a bachelor\'s degree or higher, 2017-21']]
# Drop PR State data
education = education[education['State'] != 'PR']
education

,State,Area name,"Less than a high school diploma, 2017-21","High school diploma only, 2017-21","Some college or associate's degree, 2017-21","Bachelor's degree or higher, 2017-21","Percent of adults with less than a high school diploma, 2017-21","Percent of adults with a high school diploma only, 2017-21","Percent of adults completing some college or associate's degree, 2017-21","Percent of adults with a bachelor's degree or higher, 2017-21"
2,AL,Autauga County,"4,126","12,977","11,367","11,144",10.4,32.8,28.7,28.1
3,AL,Baldwin County,"14,555","44,342","50,518","52,562",9.0,27.4,31.2,32.5
4,AL,Barbour County,"4,378","6,600","5,010","2,007",24.3,36.7,27.8,11.2
5,AL,Bibb County,"3,125","7,052","3,967","1,913",19.5,43.9,24.7,11.9
6,AL,Blount County,"6,650","14,284","13,673","6,061",16.4,35.1,33.6,14.9
...,...,...,...,...,...,...,...,...,...,...
3201,WY,Sweetwater County,"2,029","9,069","11,324","5,394",7.3,32.6,40.7,19.4
3202,WY,Teton County,701,"2,984","3,755","10,219",4.0,16.9,21.3,57.9
3203,WY,Uinta County,851,"4,820","5,186","2,376",6.4,36.4,39.2,18.0
3204,WY,Washakie County,320,"1,391","2,293","1,419",5.9,25.7,42.3,26.2


In [18]:
education.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3143 entries, 2 to 3205
Data columns (total 10 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   State                                                                     3143 non-null   object 
 1   Area name                                                                 3143 non-null   object 
 2   Less than a high school diploma, 2017-21                                  3136 non-null   object 
 3   High school diploma only, 2017-21                                         3136 non-null   object 
 4   Some college or associate's degree, 2017-21                               3135 non-null   object 
 5   Bachelor's degree or higher, 2017-21                                      3135 non-null   object 
 6   Percent of adults with less than a high school diploma, 2017-21 

Lets now check out for any NaN rows

In [19]:
education[education.isna().any(axis=1)]

,State,Area name,"Less than a high school diploma, 2017-21","High school diploma only, 2017-21","Some college or associate's degree, 2017-21","Bachelor's degree or higher, 2017-21","Percent of adults with less than a high school diploma, 2017-21","Percent of adults with a high school diploma only, 2017-21","Percent of adults completing some college or associate's degree, 2017-21","Percent of adults with a bachelor's degree or higher, 2017-21"
96,AK,Prince of Wales-Outer Ketchikan Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,AK,Skagway-Hoonah-Angoon Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,AK,Valdez-Cordova Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,AK,Wrangell-Petersburg Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1691,MT,Yellowstone National Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2708,TX,Kenedy County,84,15,NaN,4,81.6,14.6,0.0,3.9
2728,TX,Loving County,3,9,40,NaN,5.8,17.3,76.9,0.0
2974,VA,Bedford city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2979,VA,Clifton Forge city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Drop the rows with NaN entirely
education = education.dropna()
education[education.isna().any(axis=1)]

,State,Area name,"Less than a high school diploma, 2017-21","High school diploma only, 2017-21","Some college or associate's degree, 2017-21","Bachelor's degree or higher, 2017-21","Percent of adults with less than a high school diploma, 2017-21","Percent of adults with a high school diploma only, 2017-21","Percent of adults completing some college or associate's degree, 2017-21","Percent of adults with a bachelor's degree or higher, 2017-21"


Now lets work with the population df

In [21]:
population.dropna(subset = ['Rural_Urban_Continuum_Code_2003'], inplace=True)
population = population[['State','Area_Name','POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']]
# Drop PR State data
population = population[population['State'] != 'PR']
population

,State,Area_Name,POP_ESTIMATE_2020,POP_ESTIMATE_2021,POP_ESTIMATE_2022
2,AL,Autauga County,"58,902","59,210","59,759"
3,AL,Baldwin County,"233,219","239,361","246,435"
4,AL,Barbour County,"24,960","24,539","24,706"
5,AL,Bibb County,"22,183","22,370","22,005"
6,AL,Blount County,"59,102","59,085","59,512"
...,...,...,...,...,...
3199,WY,Sweetwater County,"42,190","41,582","41,345"
3200,WY,Teton County,"23,377","23,622","23,287"
3201,WY,Uinta County,"20,457","20,655","20,712"
3202,WY,Washakie County,"7,658","7,712","7,719"


In [22]:
population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3136 entries, 2 to 3203
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   State              3136 non-null   object
 1   Area_Name          3136 non-null   object
 2   POP_ESTIMATE_2020  3128 non-null   object
 3   POP_ESTIMATE_2021  3128 non-null   object
 4   POP_ESTIMATE_2022  3128 non-null   object
dtypes: object(5)
memory usage: 147.0+ KB


In [23]:
population[population.isna().any(axis=1)]

,State,Area_Name,POP_ESTIMATE_2020,POP_ESTIMATE_2021,POP_ESTIMATE_2022
317,CT,Fairfield County,NaN,NaN,NaN
318,CT,Hartford County,NaN,NaN,NaN
319,CT,Litchfield County,NaN,NaN,NaN
320,CT,Middlesex County,NaN,NaN,NaN
321,CT,New Haven County,NaN,NaN,NaN
322,CT,New London County,NaN,NaN,NaN
323,CT,Tolland County,NaN,NaN,NaN
324,CT,Windham County,NaN,NaN,NaN


We don't want to exclude all of Connecticut so we will manually enter the population from a different source.

Source: https://portal.ct.gov/DPH/Health-Information-Systems--Reporting/Population/Annual-Town-and-County-Population-for-Connecticut

In [24]:
population.loc[population[population['Area_Name'] == 'Fairfield County'].index[0], ['POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']] = [957050, 959768, 956446]
population.loc[population[population['Area_Name'] == 'Hartford County'].index[0], ['POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']] = [898682, 896854, 898636]
population.loc[population[population['Area_Name'] == 'Litchfield County'].index[0], ['POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']] = [184938, 185000, 185175]
population.loc[population[population['Area_Name'] == 'Middlesex County'].index[0], ['POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']] = [164063, 164759, 164568]
population.loc[population[population['Area_Name'] == 'New Haven County'].index[0], ['POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']] = [864094, 863700, 864751]
population.loc[population[population['Area_Name'] == 'New London County'].index[0], ['POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']] = [268450, 268805, 269131]
population.loc[population[population['Area_Name'] == 'Tolland County'].index[0], ['POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']] = [149767, 150293, 150120]
population.loc[population[population['Area_Name'] == 'Windham County'].index[0], ['POP_ESTIMATE_2020', 'POP_ESTIMATE_2021', 'POP_ESTIMATE_2022']] = [116404, 116418, 116503]

population[population['State'] == 'CT']
# Drop the rows with NaN entirely
#population = population.dropna()
#population[population.isna().any(axis=1)]

,State,Area_Name,POP_ESTIMATE_2020,POP_ESTIMATE_2021,POP_ESTIMATE_2022
317,CT,Fairfield County,957050,959768,956446
318,CT,Hartford County,898682,896854,898636
319,CT,Litchfield County,184938,185000,185175
320,CT,Middlesex County,164063,164759,164568
321,CT,New Haven County,864094,863700,864751
322,CT,New London County,268450,268805,269131
323,CT,Tolland County,149767,150293,150120
324,CT,Windham County,116404,116418,116503


Finally, apply same steps to poverty df

In [25]:
poverty.dropna(subset = ['Rural-urban_Continuum_Code_2003'], inplace=True)
poverty = poverty[['State','Area_name','POVALL_2021']]
# Drop PR State data
poverty = poverty[poverty['State'] != 'PR']
poverty

,State,Area_name,POVALL_2021
2,AL,Autauga County,6296.0
3,AL,Baldwin County,25526.0
4,AL,Barbour County,5089.0
5,AL,Bibb County,4204.0
6,AL,Blount County,6992.0
...,...,...,...
3190,WY,Sweetwater County,3761.0
3191,WY,Teton County,1392.0
3192,WY,Uinta County,1917.0
3193,WY,Washakie County,773.0


In [26]:
poverty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3136 entries, 2 to 3194
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   State        3136 non-null   object 
 1   Area_name    3136 non-null   object 
 2   POVALL_2021  3135 non-null   float64
dtypes: float64(1), object(2)
memory usage: 98.0+ KB


In [27]:
poverty[poverty.isna().any(axis=1)]

,State,Area_name,POVALL_2021
562,HI,Kalawao County,NaN


In [28]:
# Drop the rows with NaN entirely
poverty = poverty.dropna()
poverty[poverty.isna().any(axis=1)]

,State,Area_name,POVALL_2021


We can now look at demographics

In [29]:
demographics.head()

,GEO_ID,NAME,P6_001N,P6_001NA,P6_002N,P6_002NA,P6_003N,P6_003NA,P6_004N,P6_004NA,P6_005N,P6_005NA,P6_006N,P6_006NA,P6_007N,P6_007NA,Unnamed: 16
0,Geography,Geographic Area Name,!!Total races tallied:,Annotation of !!Total races tallied:,!!Total races tallied:!!White alone or in com...,Annotation of !!Total races tallied:!!White a...,!!Total races tallied:!!Black or African Amer...,Annotation of !!Total races tallied:!!Black o...,!!Total races tallied:!!American Indian and A...,Annotation of !!Total races tallied:!!America...,!!Total races tallied:!!Asian alone or in com...,Annotation of !!Total races tallied:!!Asian a...,!!Total races tallied:!!Native Hawaiian and O...,Annotation of !!Total races tallied:!!Native ...,!!Total races tallied:!!Some Other Race alone...,Annotation of !!Total races tallied:!!Some Ot...,NaN
1,0500000US01001,"Autauga County, Alabama",62152,NaN,45084,NaN,12266,NaN,1426,NaN,1360,NaN,115,NaN,1901,NaN,NaN
2,0500000US01003,"Baldwin County, Alabama",247467,NaN,203968,NaN,20913,NaN,6961,NaN,3498,NaN,354,NaN,11773,NaN,NaN
3,0500000US01005,"Barbour County, Alabama",25973,NaN,11939,NaN,12261,NaN,362,NaN,152,NaN,13,NaN,1246,NaN,NaN
4,0500000US01007,"Bibb County, Alabama",23121,NaN,17275,NaN,4643,NaN,430,NaN,56,NaN,18,NaN,699,NaN,NaN


We need to split the name column into a county and state as well as convert our columns from totals to percentages while renaming the columns as well.

In [30]:
demographics[['Name', 'State']] = demographics['NAME'].str.split(', ', expand=True)
demographics = demographics.iloc[1:]
demographics[['P6_001N', 'P6_002N', 'P6_003N', 'P6_004N', 'P6_005N', 'P6_006N']] = demographics[['P6_001N', 'P6_002N', 'P6_003N', 'P6_004N', 'P6_005N', 'P6_006N']].astype(int)
demographics[['P6_002N', 'P6_003N', 'P6_004N', 'P6_005N', 'P6_006N']] = demographics[['P6_002N', 'P6_003N', 'P6_004N', 'P6_005N', 'P6_006N']].div(demographics['P6_001N'], axis=0) * 100
# df[['A', 'B', 'C']] = df[['A', 'B', 'C']].div(df['D'], axis=0)
demographics = demographics[['Name', 'State', 'P6_002N', 'P6_003N', 'P6_004N', 'P6_005N', 'P6_006N']]

demographics.rename(columns={'P6_002N': 'Percent Caucasian','P6_003N':'Percent African American','P6_004N':'Percent Indian',
                             'P6_005N':'Percent Asian', 'P6_006N':'Percent Pacific'},inplace=True)

demographics.head()

,Name,State,Percent Caucasian,Percent African American,Percent Indian,Percent Asian,Percent Pacific
1,Autauga County,Alabama,72.538293,19.735487,2.294375,2.188184,0.185030
2,Baldwin County,Alabama,82.422303,8.450824,2.812900,1.413522,0.143049
3,Barbour County,Alabama,45.966966,47.206715,1.393755,0.585223,0.050052
4,Bibb County,Alabama,74.715626,20.081311,1.859781,0.242204,0.077851
5,Blount County,Alabama,86.188161,1.985829,2.900899,0.568741,0.104852


Now we merge all data sources together. We need to convert our state column from abbreviated state labels to actual state names.

In [31]:
county_stats = pd.merge(population, education, left_on=['State', 'Area_Name'], right_on=['State', 'Area name'])
county_stats = pd.merge(county_stats, poverty, left_on=['State', 'Area name'], right_on=['State', 'Area_name'])
#county_stats = pd.merge(county_stats, demographics, left_on=['State', 'Area name'], right_on=['State', 'Name'])
states = {
        'AL': 'Alabama',
        'AK': 'Alaska',
        'AZ': 'Arizona',
        'AR': 'Arkansas',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DE': 'Delaware',
        'DC': 'District of Columbia',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'IA': 'Iowa',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'ME': 'Maine',
        'MD': 'Maryland',
        'MA': 'Massachusetts',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MS': 'Mississippi',
        'MO': 'Missouri',
        'MT': 'Montana',
        'NE': 'Nebraska',
        'NV': 'Nevada',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NY': 'New York',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VT': 'Vermont',
        'VA': 'Virginia',
        'WA': 'Washington',
        'WV': 'West Virginia',
        'WI': 'Wisconsin',
        'WY': 'Wyoming',
      }
county_stats['State'] = county_stats['State'].replace(states)

county_stats = pd.merge(county_stats, demographics, left_on=['State', 'Area name'], right_on=['State', 'Name'])
county_stats

,State,Area_Name,POP_ESTIMATE_2020,POP_ESTIMATE_2021,POP_ESTIMATE_2022,Area name,"Less than a high school diploma, 2017-21","High school diploma only, 2017-21","Some college or associate's degree, 2017-21","Bachelor's degree or higher, 2017-21",...,"Percent of adults completing some college or associate's degree, 2017-21","Percent of adults with a bachelor's degree or higher, 2017-21",Area_name,POVALL_2021,Name,Percent Caucasian,Percent African American,Percent Indian,Percent Asian,Percent Pacific
0,Alabama,Autauga County,"58,902","59,210","59,759",Autauga County,"4,126","12,977","11,367","11,144",...,28.7,28.1,Autauga County,6296.0,Autauga County,72.538293,19.735487,2.294375,2.188184,0.185030
1,Alabama,Baldwin County,"233,219","239,361","246,435",Baldwin County,"14,555","44,342","50,518","52,562",...,31.2,32.5,Baldwin County,25526.0,Baldwin County,82.422303,8.450824,2.812900,1.413522,0.143049
2,Alabama,Barbour County,"24,960","24,539","24,706",Barbour County,"4,378","6,600","5,010","2,007",...,27.8,11.2,Barbour County,5089.0,Barbour County,45.966966,47.206715,1.393755,0.585223,0.050052
3,Alabama,Bibb County,"22,183","22,370","22,005",Bibb County,"3,125","7,052","3,967","1,913",...,24.7,11.9,Bibb County,4204.0,Bibb County,74.715626,20.081311,1.859781,0.242204,0.077851
4,Alabama,Blount County,"59,102","59,085","59,512",Blount County,"6,650","14,284","13,673","6,061",...,33.6,14.9,Blount County,6992.0,Blount County,86.188161,1.985829,2.900899,0.568741,0.104852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116,Wyoming,Sweetwater County,"42,190","41,582","41,345",Sweetwater County,"2,029","9,069","11,324","5,394",...,40.7,19.4,Sweetwater County,3761.0,Sweetwater County,82.504071,1.429245,3.167052,1.317819,0.370704
3117,Wyoming,Teton County,"23,377","23,622","23,287",Teton County,701,"2,984","3,755","10,219",...,21.3,57.9,Teton County,1392.0,Teton County,83.373748,0.500626,1.630945,2.303661,0.140801
3118,Wyoming,Uinta County,"20,457","20,655","20,712",Uinta County,851,"4,820","5,186","2,376",...,39.2,18.0,Uinta County,1917.0,Uinta County,88.286004,0.783699,2.825927,1.000369,0.368800
3119,Wyoming,Washakie County,"7,658","7,712","7,719",Washakie County,320,"1,391","2,293","1,419",...,42.3,26.2,Washakie County,773.0,Washakie County,86.147916,0.602265,2.372922,0.831125,0.192725


Rename columns for clarity

In [32]:
county_stats.drop(columns=['Area name', 'Area_name', 'Name'], inplace=True)
county_stats.rename(columns={'State': 'state','Area_Name':'county','POP_ESTIMATE_2020':'pop_2020',
                             'POP_ESTIMATE_2021':'pop_2021', 'POP_ESTIMATE_2022':'pop_2022',
                             'Less than a high school diploma, 2017-21':'Less than a high school diploma',
                             'High school diploma only, 2017-21':'High school diploma only',
                             'Some college or associate\'s degree, 2017-21':'Some college or associate\'s degree',
                             'Bachelor\'s degree or higher, 2017-21':'Bachelor\'s degree or higher',
                             'Percent of adults with less than a high school diploma, 2017-21':'Percent of adults with less than a high school diploma',
                             'Percent of adults with a high school diploma only, 2017-21':'Percent of adults with a high school diploma only',
                             'Percent of adults completing some college or associate\'s degree, 2017-21':'Percent of adults completing some college or associate\'s degree',
                             'Percent of adults with a bachelor\'s degree or higher, 2017-21':'Percent of adults with a bachelor\'s degree or higher',
                             'POVALL_2021':'pov_2021'},inplace=True)


county_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3121 entries, 0 to 3120
Data columns (total 19 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   state                                                            3121 non-null   object 
 1   county                                                           3121 non-null   object 
 2   pop_2020                                                         3121 non-null   object 
 3   pop_2021                                                         3121 non-null   object 
 4   pop_2022                                                         3121 non-null   object 
 5   Less than a high school diploma                                  3121 non-null   object 
 6   High school diploma only                                         3121 non-null   object 
 7   Some college or associate's degree        

In [33]:
# Remove names that do not match between county_data and county_stats
# First fix names from county_stats to find better matches
county_stats['county'] = county_stats['county'].str.replace(' County', '')
county_stats['county'] = county_stats['county'].str.replace(' Parish', '')
county_stats['county'] = county_stats['county'].str.replace(' Borough', '')
# and fix county_data
county_data['county'] = county_data['county'].str.replace(' Borough', '')
county_data['county'] = county_data['county'].str.replace('New York City', 'New York')


county_stats.info()
county_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3121 entries, 0 to 3120
Data columns (total 19 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   state                                                            3121 non-null   object 
 1   county                                                           3121 non-null   object 
 2   pop_2020                                                         3121 non-null   object 
 3   pop_2021                                                         3121 non-null   object 
 4   pop_2022                                                         3121 non-null   object 
 5   Less than a high school diploma                                  3121 non-null   object 
 6   High school diploma only                                         3121 non-null   object 
 7   Some college or associate's degree        

In [34]:
stat_strings = county_stats['county'].unique()
data_strings = county_data['county'].unique()
matches1 = [str1 for str1 in stat_strings for str2 in data_strings if str2 in str1]
matches2 = [str1 for str1 in data_strings for str2 in stat_strings if str2 in str1]
county_stats = county_stats[county_stats['county'].str.contains('|'.join(matches1))]
county_data = county_data[county_data['county'].str.contains('|'.join(matches2))]
county_stats.info()
county_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3119 entries, 0 to 3120
Data columns (total 19 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   state                                                            3119 non-null   object 
 1   county                                                           3119 non-null   object 
 2   pop_2020                                                         3119 non-null   object 
 3   pop_2021                                                         3119 non-null   object 
 4   pop_2022                                                         3119 non-null   object 
 5   Less than a high school diploma                                  3119 non-null   object 
 6   High school diploma only                                         3119 non-null   object 
 7   Some college or associate's degree        

The last step we want to do is fix our column types. 

In [35]:
county_stats[['pop_2020', 'pop_2021', 'pop_2022', 'Less than a high school diploma', 'High school diploma only', 'Some college or associate\'s degree', 'Bachelor\'s degree or higher']] = county_stats[['pop_2020', 'pop_2021', 'pop_2022', 'Less than a high school diploma', 'High school diploma only', 'Some college or associate\'s degree', 'Bachelor\'s degree or higher']].astype(str).replace(',', '', regex=True).astype(int)
county_stats.head()

C:\Users\matth\AppData\Local\Temp\ipykernel_5956\599054344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_stats[['pop_2020', 'pop_2021', 'pop_2022', 'Less than a high school diploma', 'High school diploma only', 'Some college or associate\'s degree', 'Bachelor\'s degree or higher']] = county_stats[['pop_2020', 'pop_2021', 'pop_2022', 'Less than a high school diploma', 'High school diploma only', 'Some college or associate\'s degree', 'Bachelor\'s degree or higher']].astype(str).replace(',', '', regex=True).astype(int)


,state,county,pop_2020,pop_2021,pop_2022,Less than a high school diploma,High school diploma only,Some college or associate's degree,Bachelor's degree or higher,Percent of adults with less than a high school diploma,Percent of adults with a high school diploma only,Percent of adults completing some college or associate's degree,Percent of adults with a bachelor's degree or higher,pov_2021,Percent Caucasian,Percent African American,Percent Indian,Percent Asian,Percent Pacific
0,Alabama,Autauga,58902,59210,59759,4126,12977,11367,11144,10.4,32.8,28.7,28.1,6296.0,72.538293,19.735487,2.294375,2.188184,0.185030
1,Alabama,Baldwin,233219,239361,246435,14555,44342,50518,52562,9.0,27.4,31.2,32.5,25526.0,82.422303,8.450824,2.812900,1.413522,0.143049
2,Alabama,Barbour,24960,24539,24706,4378,6600,5010,2007,24.3,36.7,27.8,11.2,5089.0,45.966966,47.206715,1.393755,0.585223,0.050052
3,Alabama,Bibb,22183,22370,22005,3125,7052,3967,1913,19.5,43.9,24.7,11.9,4204.0,74.715626,20.081311,1.859781,0.242204,0.077851
4,Alabama,Blount,59102,59085,59512,6650,14284,13673,6061,16.4,35.1,33.6,14.9,6992.0,86.188161,1.985829,2.900899,0.568741,0.104852


We want to also add density and average house size

In [36]:
# ADD ANOTHER FILE
county_density = pd.read_csv('../data/Average_Household_Size_and_Population_Density_-_County.csv')
county_density = county_density[['State', 'NAME', 'B25010_001E', 'B01001_calc_PopDensity']]
county_density = county_density.rename(columns={'State': 'state', 'NAME': 'county', 'B25010_001E': 'avg_house_size', 'B01001_calc_PopDensity': 'pop_density'})

# MERGE
county_stats = county_stats.loc[county_stats['county'] != 'Bristol Bay']
county_stats = county_stats.loc[county_stats['county'] != 'Lake and Peninsula']
index_drop = county_stats[(county_stats['state'] == 'New York') & ((county_stats['county'] == 'Richmond') | (county_stats['county'] == 'Kings'))].index
county_stats.drop(index_drop , inplace=True)

county_density['county'] = county_density['county'].str.replace(' County', '')
county_density['county'] = county_density['county'].str.replace(' Borough', '')
county_density['county'] = county_density['county'].str.replace(' Parish', '')

county_stats = pd.merge(county_stats, county_density, how='inner', left_on=['state', 'county'], right_on=['state','county'])
county_stats.head()

,state,county,pop_2020,pop_2021,pop_2022,Less than a high school diploma,High school diploma only,Some college or associate's degree,Bachelor's degree or higher,Percent of adults with less than a high school diploma,...,Percent of adults completing some college or associate's degree,Percent of adults with a bachelor's degree or higher,pov_2021,Percent Caucasian,Percent African American,Percent Indian,Percent Asian,Percent Pacific,avg_house_size,pop_density
0,Alabama,Autauga,58902,59210,59759,4126,12977,11367,11144,10.4,...,28.7,28.1,6296.0,72.538293,19.735487,2.294375,2.188184,0.185030,2.59,35.853419
1,Alabama,Baldwin,233219,239361,246435,14555,44342,50518,52562,9.0,...,31.2,32.5,25526.0,82.422303,8.450824,2.812900,1.413522,0.143049,2.61,50.541504
2,Alabama,Barbour,24960,24539,24706,4378,6600,5010,2007,24.3,...,27.8,11.2,5089.0,45.966966,47.206715,1.393755,0.585223,0.050052,2.49,11.247981
3,Alabama,Bibb,22183,22370,22005,3125,7052,3967,1913,19.5,...,24.7,11.9,4204.0,74.715626,20.081311,1.859781,0.242204,0.077851,2.99,13.973114
4,Alabama,Blount,59102,59085,59512,6650,14284,13673,6061,16.4,...,33.6,14.9,6992.0,86.188161,1.985829,2.900899,0.568741,0.104852,2.77,34.515816


We want to also manually edit the NYC 

In [37]:
county_stats.loc[county_stats[county_stats['county'] == 'New York'].index[0], ['pop_2020', 'pop_2021', 'pop_2022', 'pop_density']] = [8772978, 8467513, 8495691, 11234]
county_stats[county_stats['county'] == 'New York']

,state,county,pop_2020,pop_2021,pop_2022,Less than a high school diploma,High school diploma only,Some college or associate's degree,Bachelor's degree or higher,Percent of adults with less than a high school diploma,...,Percent of adults completing some college or associate's degree,Percent of adults with a bachelor's degree or higher,pov_2021,Percent Caucasian,Percent African American,Percent Indian,Percent Asian,Percent Pacific,avg_house_size,pop_density
1836,New York,New York,8772978,8467513,8495691,148767,156704,174128,801081,11.6,...,13.6,62.6,264938.0,52.209472,15.119599,1.426146,13.48569,0.289279,2.07,11234.0


Save data for Exploratory Data Analysis

In [38]:
datapath = '../data'
save_file(county_data, 'county_data_cleaned.csv', datapath)
save_file(county_stats, 'county_stats_cleaned.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../data\county_data_cleaned.csv"
A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../data\county_stats_cleaned.csv"
